# Carga del dataset *Climate Model Simulation Crashes*

Para comenzar con esta actividad, realizamos la carga del *Climate Model Simulation Crashes Data Set* disponible en la plataforma *UCI Machine Learning Repository* (https://archive.ics.uci.edu/ml/datasets/climate+model+simulation+crashes)

Comenzamos realizando la carga mediante ``pandas``. En este caso, el archivo proporcionado contiene una cabecera con los identificadores de cada columna. Sin embargo, el fichero presentaba una dificultad adicional y es que cada columna estaba separada por un número aparentemente arbitrario de espacios, lo que complicaba su lectura. Para solventar este error, se abrió dicho fichero desde *VS Code* y, mediante el uso de la expresión regular " +", se reemplazaron todos los espacios por una coma.

In [1]:
import pandas as pd

filename = 'pop_failures.dat'
df = pd.read_csv(filename)
print(f'Shape -> {df.shape}')
df.head()

Shape -> (540, 21)


,Study,Run,vconst_corr,vconst_2,vconst_3,vconst_4,vconst_5,vconst_7,ah_corr,ah_bolus,...,efficiency_factor,tidal_mix_max,vertical_decay_scale,convect_corr,bckgrnd_vdc1,bckgrnd_vdc_ban,bckgrnd_vdc_eq,bckgrnd_vdc_psim,Prandtl,outcome
0,1,1,0.859036,0.927825,0.252866,0.298838,0.170521,0.735936,0.428325,0.567947,...,0.245675,0.104226,0.869091,0.997518,0.448620,0.307522,0.858310,0.796997,0.869893,0
1,1,2,0.606041,0.457728,0.359448,0.306957,0.843331,0.934851,0.444572,0.828015,...,0.616870,0.975786,0.914344,0.845247,0.864152,0.346713,0.356573,0.438447,0.512256,1
2,1,3,0.997600,0.373238,0.517399,0.504993,0.618903,0.605571,0.746225,0.195928,...,0.679355,0.803413,0.643995,0.718441,0.924775,0.315371,0.250642,0.285636,0.365858,1
3,1,4,0.783408,0.104055,0.197533,0.421837,0.742056,0.490828,0.005525,0.392123,...,0.471463,0.597879,0.761659,0.362751,0.912819,0.977971,0.845921,0.699431,0.475987,1
4,1,5,0.406250,0.513199,0.061812,0.635837,0.844798,0.441502,0.191926,0.487546,...,0.551543,0.743877,0.312349,0.650223,0.522261,0.043545,0.376660,0.280098,0.132283,1


# Conteo de clases y desequilibrado

Procedemos a hacer un conteo de las diferentes clases en el dataset haciendo uso del método ``.value_counts()``.

In [2]:
df['outcome'].value_counts()

1    494
0     46
Name: outcome, dtype: int64

Como podemos observar, el dataset cuenta con un desequilibrado notable de los datos. En concreto, contamos con 540 datos, el ~91.48% etiquetados como "1" y el resto como "0".

# Segmentado en conjunto de entrenamiento y conjunto de test

En este caso, vamos a realizar una validación cruzada para determinar el rendimiento de los modelos a entrenar. Por tanto a continuación únicamente separaremos variables de entrada y variable objetivo. 

In [3]:
X = df[df.columns[:-1]]
Y = df['outcome']

# Normalizado de las variables de entrada numéricas

Para realizar el normalizado de las variables, haremos uso de ``RobustScaler`` que, como ya vimos en lecciones anteriores, permite realizar un normalizado robusto a posibles *outliers* dentro de nuestro DataSet. Utilizaremos además la función ``fit_transform()`` para realizar el normalizado en un solo paso. Sin embargo, antes de proceder, eliminaremos las variables correspondientes a las dos primeras columnas del DataFrame.

In [4]:
X = X.drop(['Study', 'Run'], axis = 1)
X

,vconst_corr,vconst_2,vconst_3,vconst_4,vconst_5,vconst_7,ah_corr,ah_bolus,slm_corr,efficiency_factor,tidal_mix_max,vertical_decay_scale,convect_corr,bckgrnd_vdc1,bckgrnd_vdc_ban,bckgrnd_vdc_eq,bckgrnd_vdc_psim,Prandtl
0,0.859036,0.927825,0.252866,0.298838,0.170521,0.735936,0.428325,0.567947,0.474370,0.245675,0.104226,0.869091,0.997518,0.448620,0.307522,0.858310,0.796997,0.869893
1,0.606041,0.457728,0.359448,0.306957,0.843331,0.934851,0.444572,0.828015,0.296618,0.616870,0.975786,0.914344,0.845247,0.864152,0.346713,0.356573,0.438447,0.512256
2,0.997600,0.373238,0.517399,0.504993,0.618903,0.605571,0.746225,0.195928,0.815667,0.679355,0.803413,0.643995,0.718441,0.924775,0.315371,0.250642,0.285636,0.365858
3,0.783408,0.104055,0.197533,0.421837,0.742056,0.490828,0.005525,0.392123,0.010015,0.471463,0.597879,0.761659,0.362751,0.912819,0.977971,0.845921,0.699431,0.475987
4,0.406250,0.513199,0.061812,0.635837,0.844798,0.441502,0.191926,0.487546,0.358534,0.551543,0.743877,0.312349,0.650223,0.522261,0.043545,0.376660,0.280098,0.132283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,0.657136,0.489375,0.133713,0.411950,0.087780,0.356289,0.480204,0.029678,0.400102,0.280546,0.384117,0.885948,0.768482,0.459479,0.334482,0.573002,0.610183,0.737706
536,0.915894,0.842720,0.518947,0.090622,0.336981,0.893576,0.978703,0.674868,0.263398,0.798108,0.353546,0.044796,0.990900,0.347027,0.512499,0.810549,0.593332,0.142565
537,0.478600,0.941185,0.769245,0.950776,0.189406,0.112743,0.745645,0.527096,0.870987,0.193103,0.829563,0.101506,0.548878,0.381966,0.198811,0.867108,0.461632,0.652817
538,0.007793,0.779287,0.867468,0.704820,0.983282,0.420303,0.710612,0.174746,0.267685,0.761134,0.436714,0.690132,0.825133,0.981656,0.113193,0.364799,0.201469,0.536535


In [5]:
from sklearn.preprocessing import RobustScaler

X = RobustScaler().fit_transform(X)
X

array([[ 0.71751352,  0.85918326, -0.49682123, ...,  0.71750992,
         0.60113527,  0.74082277],
       [ 0.21191974, -0.08400007, -0.28264478, ..., -0.28828816,
        -0.1220402 ,  0.02564715],
       [ 0.99442345, -0.25351739,  0.03475513, ..., -0.50064096,
        -0.43025281, -0.26710902],
       ...,
       [-0.04276343,  0.8859888 ,  0.54083458, ...,  0.7351464 ,
        -0.07527832,  0.30673092],
       [-0.9836394 ,  0.56116406,  0.73821232, ..., -0.27179948,
        -0.60001161,  0.07419914],
       [ 0.21598363, -0.93905727,  0.19725104, ..., -0.07227361,
         0.52720718,  0.52594499]])

Una vez nos hemos asegurado que todas las variables del conjunto de entrenamiento son de tipo numérico, podemos comenzar a entrenar nuestro modelo SVM.

# Evaluación del rendimento de un modelo SVM con diferentes parámetros

A continuación vamos a realizar un entrenamiento de varios modelos de tipo SVM con diferentes parámetros. En concreto estudiaremos los siguientes tipos de parámetro:

- Función de kernel a utilizar: Lineal, Polinómico, Gaussiano y Sigmoide
- Valor del parámetro ``C`` de regularización.
- El parámetro ``gamma`` en los tipos de Kernel que lo admitan.

Además, debemos tener en cuenta que nuestro dataset se encuentra desequilibrado, por lo que utilizaremos el parámetro ``class_weight = balanced``. De nuevo, como en lecciones anteriores haremos uso de la ``balanced_accuracy`` como métrica del rendimiento, pues proporciona una intuición correcta de la precisión de clasificación en cada clase. En concreto, usaremos validación cruzada de *Monte Carlo*.

In [6]:
import numpy as np
import random
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import cross_val_score, ShuffleSplit

# Comenzamos definiendo los parámetros que vamos a usar
# Posibles valores de la función de Kernel
kernels = ['poly', 'rbf', 'sigmoid']
# Posibles valores del parámetro de regularización C
# Valores pequeños -> Se toleran más errores de clasificación, menos overfitting
# Valores grandes -> Se toleran menos errores de clasficación, mayor overfitting
Cs = [0.1, 1, 10, 100, 1000]
# Posibles valores del parámetro gamma
gammas = [1, 0.1, 0.001, 0.001]

# Incializamos una semilla para poder replicar el reparto
seed = random.randint(0, 10)
monte_carlo = ShuffleSplit(n_splits = 100, test_size = 0.3, random_state = seed)

# Comenzamos entrenando todos los modelos, generaremos un Array 3-D que contendrá
# los resultados para cada tipo de kernel, con cada valor de C y Gamma.
baccs = [[[]]]
i, j = 0, 0
# Iteramos sobre los kernels
for kernel in kernels:
    # Iteramos sobre las Cs
    for c in Cs:
        # Iteramos sobre los valores de Gamma
        for gamma in gammas:
            # Entrenamos el modelo con la combinación de parámetros
            model_temp = SVC(C = c, kernel = kernel, gamma = gamma, class_weight = 'balanced')
            # Obtenemos el valor de Balanced Accuracy mediante validación cruzada
            # con Monte Carlo
            res_mc_temp = cross_val_score(model_temp, X, Y, scoring = 'balanced_accuracy')
            # Almacenamos la media de las balanced accuracy obtenidas
            baccs[i][j].append(res_mc_temp.mean())
        baccs[i].append([])
        j = j + 1
    baccs[-1].pop()
    baccs.append([])
    baccs[-1].append([])
    i, j = i + 1, 0
baccs.pop()

# Realizamos el mismo experimento con kernel = 'linear' pero sin gamma
baccs_linear = []
for c in Cs:
    model_temp = SVC(C = c, kernel = 'linear', class_weight = 'balanced')
    res_mc_temp = cross_val_score(model_temp, X, Y, scoring = 'balanced_accuracy')
    baccs_linear.append(res_mc_temp.mean())


# Obtenemos la posición de la máxima Balanced Accuracy en el Array 3-D generado
baccs_aux = np.array(baccs)
i, j, k = np.unravel_index(baccs_aux.argmax(), baccs_aux.shape)
# Mostramos los parámetros óptimos para el experimento
print('--------------------------------------------------------')
print(f'CONFIGURACION OPTIMA\nBalanced Accuracy -> {baccs_aux.max()}')
print(f'Kernel -> {kernels[i]}\nC -> {Cs[j]}\nGamma -> {gammas[k]}')
print('--------------------------------------------------------')

# Obtenemos lo mismo para el caso de linear
baccs_linear_aux = np.array(baccs_linear)
i = np.argmax(baccs_linear_aux)
print('--------------------------------------------------------')
print(f'CONFIGURACION OPTIMA LINEAR\nBalanced Accuracy -> {baccs_linear_aux.max()}')
print(f'Kernel -> linear\nC -> {Cs[i]}')
print('--------------------------------------------------------')

--------------------------------------------------------
CONFIGURACION OPTIMA
Balanced Accuracy -> 0.8869490826633685
Kernel -> rbf
C -> 1
Gamma -> 0.1
--------------------------------------------------------
--------------------------------------------------------
CONFIGURACION OPTIMA LINEAR
Balanced Accuracy -> 0.8637662337662337
Kernel -> linear
C -> 1000
--------------------------------------------------------


In [7]:
baccs_aux

array([[[0.65321171, 0.57591837, 0.57591837, 0.57591837],
        [0.65321171, 0.71361575, 0.5       , 0.5       ],
        [0.65321171, 0.65423212, 0.5       , 0.5       ],
        [0.65321171, 0.65321171, 0.69713049, 0.69713049],
        [0.65321171, 0.65321171, 0.5       , 0.5       ]],

       [[0.5       , 0.87267368, 0.55714286, 0.55714286],
        [0.5       , 0.88694908, 0.82744795, 0.82744795],
        [0.5       , 0.78736343, 0.87067409, 0.87067409],
        [0.5       , 0.79736343, 0.87487734, 0.87487734],
        [0.5       , 0.79736343, 0.86174603, 0.86174603]],

       [[0.84018759, 0.86546073, 0.61673882, 0.61673882],
        [0.79973201, 0.85654298, 0.5       , 0.5       ],
        [0.79165121, 0.88066378, 0.87153164, 0.87153164],
        [0.79065141, 0.87067409, 0.85960421, 0.85960421],
        [0.80883323, 0.86250258, 0.85164502, 0.85164502]]])

In [8]:
baccs_linear_aux

array([0.85960421, 0.85164502, 0.86073593, 0.86073593, 0.86376623])

De las salidas anteriores podemos ver como el valor óptimo de los parámetros es:

- Kernel: RBF
- C = 1
- $\gamma$ = 0.1

ofriciendo una *balanced accuracy* del ~88.7%. Otros aspectos relevantes a comentar son los siguientes:

- El kernel polinómico ofrece unos resultados claramente inferiores para este problema.
- RBF y Sigmoid ofrecen un rendimento similar.
- Valores bajos de C y altos en $\gamma$ parecen empeorar el valor de la precisión para las tres funciones de Kernel.
- El kernel lineal ofrece una precisión ~85% independientemente del valor de C pero el mejor valor se obtiene con C = 1000.